In [5]:
import os
import cv2
import numpy as np

# 데이터 로드 함수
def load_handwritten_images(image_dir, target_size=(64, 64)):
    images = []
    labels = []
    file_names = sorted(os.listdir(image_dir))
    for file_name in file_names:
        if file_name.endswith(".jpg"):
            try:
                img_path = os.path.join(image_dir, file_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Warning: Unable to read {img_path}")
                    continue
                img = cv2.resize(img, target_size) / 255.0  # 픽셀 정규화
                images.append(img)
                label = os.path.splitext(file_name)[0]  # splittext 오타 수정
                labels.append(label)
            except Exception as e:
                print(f"Error processing file {file_name}: {e}")
    # NumPy 배열 변환 및 채널 차원 추가
    images_array = np.expand_dims(np.array(images), axis=-1)
    return images_array, labels

# 손글씨 데이터 경로
handwritten_path = "/Users/kimeunsur/2024winter/4주차/font_images/seungmin_hw"

# 손글씨 데이터 로드
X_handwritten, handwritten_labels = load_handwritten_images(handwritten_path)
print(f"Handwritten data shape: {X_handwritten.shape}")
print(f"Number of labels: {len(handwritten_labels)}")


Handwritten data shape: (200, 64, 64, 1)
Number of labels: 200


In [3]:
from tensorflow.keras.models import load_model

# 사전학습 모델 로드

pretrained_model_path = "/Users/kimeunsur/2024winter/4주차/font_ai_model/ai_model/src/cnn.ipynb"
pretrained_model = load_model(pretrained_model_path)

# 기존 모델의 Encoder-Decoder 구조 복사
encoder_input = pretrained_model.input[0]
style_input = pretrained_model.input[1]
encoded = pretrained_model.get_layer("max_pooling2d_1").output  # 인코더 마지막 출력

# 디코더 부분만 재구성 (Fine-Tuning용)
from tensorflow.keras import layers, Model

x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

# Fine-Tuning 모델 생성
fine_tuned_model = Model([encoder_input, style_input], decoded)
fine_tuned_model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])


ValueError: File format not supported: filepath=/Users/kimeunsur/2024winter/4주차/font_ai_model/ai_model/src/cnn.ipynb. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(/Users/kimeunsur/2024winter/4주차/font_ai_model/ai_model/src/cnn.ipynb, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [ ]:
# 원-핫 인코딩된 스타일 정보 생성 (단일 손글씨 스타일로 가정)
from tensorflow.keras.utils import to_categorical

style_label = 0  # 손글씨 스타일 번호
style_labels = np.full((len(X_handwritten),), style_label)
style_one_hot = to_categorical(style_labels, num_classes=1)  # 단일 클래스

# 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(X_handwritten, test_size=0.2, random_state=42)

# 모델 학습
history = fine_tuned_model.fit(
    [X_train, style_one_hot[:len(X_train)]],  # 입력: 이미지와 스타일
    X_train,                                 # 출력: 재구성된 이미지
    validation_data=([X_val, style_one_hot[len(X_train):]], X_val),
    epochs=50,
    batch_size=16
)


In [ ]:
import matplotlib.pyplot as plt

# 검증 데이터에서 예측 결과 생성
reconstructed_images = fine_tuned_model.predict([X_val, style_one_hot[len(X_train):]])

# 결과 시각화
n = 5  # 시각화할 샘플 수
plt.figure(figsize=(15, 5))
for i in range(n):
    # 원본 이미지
    plt.subplot(2, n, i + 1)
    plt.imshow(X_val[i].squeeze(), cmap='gray')
    plt.title("Original")
    plt.axis('off')
    
    # 복원된 이미지
    plt.subplot(2, n, i + 1 + n)
    plt.imshow(reconstructed_images[i].squeeze(), cmap='gray')
    plt.title("Reconstructed")
    plt.axis('off')
plt.show()
